In [12]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/instagram-fake-spammer-genuine-accounts/train.csv
/kaggle/input/instagram-fake-spammer-genuine-accounts/test.csv


In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

import keras
from sklearn.preprocessing import StandardScaler , LabelEncoder
from keras.models import Model, Sequential
from keras.layers import Dense, Activation, Flatten, BatchNormalization, Dropout

In [14]:
train_path = '../input/instagram-fake-spammer-genuine-accounts/train.csv'
test_path = '../input/instagram-fake-spammer-genuine-accounts/test.csv'

train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

In [15]:
train.head(10)

,profile pic,nums/length username,fullname words,nums/length fullname,name==username,description length,external URL,private,#posts,#followers,#follows,fake
0,1,0.27,0,0.0,0,53,0,0,32,1000,955,0
1,1,0.00,2,0.0,0,44,0,0,286,2740,533,0
2,1,0.10,2,0.0,0,0,0,1,13,159,98,0
3,1,0.00,1,0.0,0,82,0,0,679,414,651,0
4,1,0.00,2,0.0,0,0,0,1,6,151,126,0
5,1,0.00,4,0.0,0,81,1,0,344,669987,150,0
6,1,0.00,2,0.0,0,50,0,0,16,122,177,0
7,1,0.00,2,0.0,0,0,0,0,33,1078,76,0
8,1,0.00,0,0.0,0,71,0,0,72,1824,2713,0
9,1,0.00,2,0.0,0,40,1,0,213,12945,813,0


In [16]:
train.tail(10)

,profile pic,nums/length username,fullname words,nums/length fullname,name==username,description length,external URL,private,#posts,#followers,#follows,fake
566,1,0.27,1,0.00,0,19,0,0,8,126,860,1
567,1,0.25,1,0.00,0,0,0,1,102,39,229,1
568,1,0.43,1,0.00,0,5,0,0,6,66,161,1
569,1,0.31,3,0.00,0,0,0,0,25,87,698,1
570,1,0.20,1,0.00,0,28,0,0,0,15,64,1
571,1,0.55,1,0.44,0,0,0,0,33,166,596,1
572,1,0.38,1,0.33,0,21,0,0,44,66,75,1
573,1,0.57,2,0.00,0,0,0,0,4,96,339,1
574,1,0.57,1,0.00,0,11,0,0,0,57,73,1
575,1,0.27,1,0.00,0,0,0,0,2,150,487,1


In [17]:
train.iloc[:]

,profile pic,nums/length username,fullname words,nums/length fullname,name==username,description length,external URL,private,#posts,#followers,#follows,fake
0,1,0.27,0,0.00,0,53,0,0,32,1000,955,0
1,1,0.00,2,0.00,0,44,0,0,286,2740,533,0
2,1,0.10,2,0.00,0,0,0,1,13,159,98,0
3,1,0.00,1,0.00,0,82,0,0,679,414,651,0
4,1,0.00,2,0.00,0,0,0,1,6,151,126,0
...,...,...,...,...,...,...,...,...,...,...,...,...
571,1,0.55,1,0.44,0,0,0,0,33,166,596,1
572,1,0.38,1,0.33,0,21,0,0,44,66,75,1
573,1,0.57,2,0.00,0,0,0,0,4,96,339,1
574,1,0.57,1,0.00,0,11,0,0,0,57,73,1


In [18]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 576 entries, 0 to 575
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   profile pic           576 non-null    int64  
 1   nums/length username  576 non-null    float64
 2   fullname words        576 non-null    int64  
 3   nums/length fullname  576 non-null    float64
 4   name==username        576 non-null    int64  
 5   description length    576 non-null    int64  
 6   external URL          576 non-null    int64  
 7   private               576 non-null    int64  
 8   #posts                576 non-null    int64  
 9   #followers            576 non-null    int64  
 10  #follows              576 non-null    int64  
 11  fake                  576 non-null    int64  
dtypes: float64(2), int64(10)
memory usage: 54.1 KB


In [19]:
x_train = train.iloc[:, :-1].values
y_train = train.iloc[:, -1].values

x_test = test.iloc[:, :-1].values
y_test = test.iloc[:, -1].values

In [20]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train_sc = sc.fit_transform(x_train)
x_test_sc = sc.transform(x_test)

In [21]:
from sklearn.decomposition import PCA 
pca = PCA(n_components=2)
x_train_sc_pca = pca.fit_transform(x_train_sc)
x_test_sc_pca = pca.transform(x_test_sc)

In [22]:
losreg = LogisticRegression(random_state = 7)
losreg.fit(x_train_sc_pca, y_train)
y_pred = losreg.predict(x_test_sc_pca)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[54  6]
 [10 50]]


0.8666666666666667

In [23]:
knn = KNeighborsClassifier(n_neighbors =9, metric = 'minkowski', p = 2)
knn.fit(x_train_sc, y_train)
y_pred = knn.predict(x_test_sc)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[55  5]
 [ 5 55]]


0.9166666666666666

In [24]:
svc = SVC(kernel = 'rbf', random_state = 5)
svc.fit(x_train_sc, y_train)
y_pred = svc.predict(x_test_sc)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[56  4]
 [10 50]]


0.8833333333333333

In [25]:
dtc = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
dtc.fit(x_train_sc, y_train)
y_pred = dtc.predict(x_test_sc)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[53  7]
 [ 3 57]]


0.9166666666666666

In [26]:
rfc = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
rfc.fit(x_train_sc, y_train)
y_pred = rfc.predict(x_test_sc)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[57  3]
 [ 5 55]]


0.9333333333333333

profile_pic = input('0 is no profile pic, 1 is profile pic is available : ')
num_by_num = input('ratio of number of numerical chars in username to its length : ')
full_name = input('full name in word tokens : ')
num_by_char = input('ratio of number of numerical characters in full name to its length : ')
name_username = input('are username and full name literally the same 0 : if not simillar and 1 : if simillar : ')
bio_len = input('bio length in characters : ')
url = input('has external URL or not 0 : dont have and 1 : have url : ')
private = input('0 : if not private , 1 : if private : ')
post = input('number of post : ')
followers = input('number of followers : ')
follows = input('number of follows : ')

In [27]:
pred1 = rfc.predict(sc.transform([[1,0,2,0,0,22,1,1,59,353,501]])) > 0.5
#pred = rfc.predict(sc.transform([[profile_pic,num_by_num,full_name,num_by_char,name_username,bio_len,url,private,post,followers,follows]])) > 0.5
if pred1 == 0:
    print('0 : genuine account')
else:
    print('1 : Spam account')

0 : genuine account
